## Example Usage of hec.unit Module ##

### Setup ###

In [25]:
import os, sys

srcdir = os.path.join(os.path.dirname(os.getcwd()), "src")
sys.path.append(srcdir)
from hec import unit  # type: ignore
from hec.unit import UnitQuantity as UQ # type: ignore
from datetime import datetime
from datetime import timedelta
from datetime import timezone
import cwms # type: ignore

### Get Some Unit Names

In [26]:
print(f"length units  = {unit.get_compatible_units('foot')}")
print(f"area units    = {unit.get_compatible_units('foot**2')}")
print(f"volumne units = {unit.get_compatible_units('foot**3')}")
print(f"flow units    = {unit.get_compatible_units('ft**3/s')}")

length units  = ['cm', 'ft', 'ftUS', 'in', 'km', 'm', 'mi', 'mm']
area units    = ['1000 m2', 'acre', 'cm2', 'ft2', 'ha', 'km2', 'm2', 'mile2']
volumne units = ['1000 m3', 'ac-ft', 'dsf', 'ft3', 'gal', 'kaf', 'kdsf', 'kgal', 'km3', 'm3', 'mcm', 'mgal', 'mile3']
flow units    = ['cfs', 'cms', 'gpm', 'KAF/mon', 'kcfs', 'kcms', 'mcm/mon', 'mgd']


### Get Aliases For a Unit

In [27]:
print(f"Aliases for ft  = {unit.get_unit_aliases('ft')}")
print(f"Aliases for ft2 = {unit.get_unit_aliases('ft2')}")
print(f"Aliases for ft3 = {unit.get_unit_aliases('ft3')}")
print(f"Aliases for cfs = {unit.get_unit_aliases('cfs')}")

Aliases for ft  = ['FEET', 'FT', 'Feet', 'feet', 'foot']
Aliases for ft2 = ['sq ft', 'square feet']
Aliases for ft3 = ['cu ft', 'cubic feet']
Aliases for cfs = ['CFS', 'FT3/S', 'FT3/SEC', 'cu-ft/sec', 'cuft/sec', 'cusecs', 'ft3/s', 'ft3/sec', 'ft^3/s']


### Scalar Unit Conversions

In [28]:
print(f"1 mile = {unit.convert_units(1, 'mi', 'ft')} feet")
print(f"1 hectare = {unit.convert_units(1, 'ha', 'acre',)} acres")

1 mile = 5280.0 feet
1 hectare = 2.4710538146716536 acres


### Converting Time Series from CWMS Python

In [29]:
elev = cwms.get_timeseries(
    ts_id="Keys.Elev.Inst.1Hour.0.Ccp-rev",
    office_id="SWT",
    unit="m",
    begin=datetime.now(timezone.utc) - timedelta(hours=6),
)
print("\n=== As Retrieved from Database ===")
print(f"Elev units     = {elev.json['units']}")
print(f"Elev values    = {[v[1] for v in elev.json['values']]}")
print(f"Vertical datum = {elev.json['vertical-datum-info']}")
print(elev.df)
print("\n=== Converting Units (in_place = False) ===")
elev2 = unit.convert_units(elev, elev.json["units"], "ft", in_place=False)
print("--- Converted Data ---")
print(f"Elev units     = {elev2.json['units']}")
print(f"Elev values    = {[v[1] for v in elev2.json['values']]}")
print(f"Vertical datum = {elev.json['vertical-datum-info']}")
print(elev2.df)
print("--- Original Data ---")
print(f"Elev units     = {elev.json['units']}")
print(f"Elev values    = {[v[1] for v in elev.json['values']]}")
print(f"Vertical datum = {elev.json['vertical-datum-info']}")
print(elev.df)
print("\n=== Converting Units (in_place = True) ===")
unit.convert_units(elev, elev.json["units"], "ft", in_place=True)
print("--- Original Data Has Been Overwritten---")
print(f"Elev units     = {elev.json['units']}")
print(f"Elev values    = {[v[1] for v in elev.json['values']]}")
print(f"Vertical datum = {elev.json['vertical-datum-info']}")
print(elev.df)


=== As Retrieved from Database ===
Elev units     = m
Elev values    = [219.465144, 219.465144, 219.465144, 219.462096, 219.462096, 219.465144]
Vertical datum = {'office': 'SWT', 'unit': 'm', 'location': 'KEYS', 'native-datum': 'NGVD-29', 'elevation': 187.522, 'offsets': [{'estimate': True, 'to-datum': 'NAVD-88', 'value': 0.1105}]}
                  date-time       value  quality-code
0 2024-10-01 14:00:00+00:00  219.465144             0
1 2024-10-01 15:00:00+00:00  219.465144             0
2 2024-10-01 16:00:00+00:00  219.465144             0
3 2024-10-01 17:00:00+00:00  219.462096             0
4 2024-10-01 18:00:00+00:00  219.462096             0
5 2024-10-01 19:00:00+00:00  219.465144             0

=== Converting Units (in_place = False) ===
--- Converted Data ---
Elev units     = ft
Elev values    = [720.0300000000001, 720.0300000000001, 720.0300000000001, 720.02, 720.02, 720.0300000000001]
Vertical datum = {'office': 'SWT', 'unit': 'm', 'location': 'KEYS', 'native-datum': 'NGVD

### Using UnitQuantity Class

In [30]:
area = UQ(20000, "acre")
depth = UQ("10 ft")
volume = area * depth
print(f"{area} * {depth} = {volume:~P}")
print(volume.to("ac-ft"))
print(volume)
print(f"{volume:6.0f~P}")
print(volume.to("dsf", in_place=True))
print(f"{volume:6.0f~P}")
volume.ito("mcm")
print(f"{volume.magnitude:6.2f} {volume.specified_units}")
print(f"Magnitude = {volume.magnitude}")
print(f"Unit = {volume.units}")
print(f"Specified unit = {volume.specified_units}")
print(f"Dimensionality = {volume.dimensionality}")
print(f"Unit aliases = {volume.getUnitAliases()}")
print(f"Compatible units = {volume.getCompatibleUnits()}")

20000 acre * 10 foot = 200000 acre·ft
200000.0 ac-ft
200000 acre * foot
200000 acre·ft
100833.33333333336 dsf
100833 d·ft³/s
246.70 mcm
Magnitude = 246.69636750950397
Unit = Mcm
Specified unit = mcm
Dimensionality = [length] ** 3
Unit aliases = ['1000000 m3']
Compatible units = ['1000 m3', 'ac-ft', 'dsf', 'ft3', 'gal', 'kaf', 'kdsf', 'kgal', 'km3', 'm3', 'mcm', 'mgal', 'mile3']
